# 0. Import Library

In [1]:
import os
if not os.path.exists("./tfdet"):
    !git clone -q http://github.com/burf/tfdetection.git
    !mv ./tfdetection/tfdet ./tfdet
    !rm -rf ./tfdetection

In [2]:
#ignore warning
import warnings, os
warnings.filterwarnings(action = "ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import tfdet

# 1. Init Dataset

In [3]:
#sample dataset
import numpy as np

image_shape = [32, 32]
sampling_size = 550
batch_size = 4

def load(x_true):
    x_true = np.random.random([*image_shape, 3]).astype(np.float32)
    return x_true
        
val_y = np.random.randint(2, size = (10, 1))
dataset = tfdet.dataset.Dataset(np.arange(10), transform = [load])
tr_pipe = val_pipe = te_pipe = tfdet.dataset.PipeLoader(dataset, batch_size = batch_size, prefetch = True)

# 2. Build Anomaly Detector

2-1. Feature-Based

In [4]:
x = tf.keras.layers.Input(shape = [*image_shape, 3])
feature = tfdet.model.backbone.wide_resnet50_2(x, weights = "imagenet_v2", indices = [0, 1, 2])
n_feature = np.sum([np.shape(f)[-1] for f in feature])
sampling_index = np.random.choice(np.arange(n_feature), sampling_size, replace = False)
out = tfdet.model.detector.padim(feature, sampling_index = sampling_index, memory_reduce = False) #align memory_reduce with train and test
model = tf.keras.Model(x, out)

extracted_feature = model.predict(tr_pipe, verbose = 1)
feature_vector = tfdet.model.train.padim.train(extracted_feature)

score, mask = tfdet.model.detector.padim(feature, feature_vector, image_shape = image_shape, sampling_index = sampling_index, memory_reduce = False) #align memory_reduce with train and test
model = tf.keras.Model(x, score)
score_pred = model.predict(val_pipe, verbose = 1)
threshold = tfdet.util.get_threshold(val_y, score_pred)
filtered_out = tfdet.model.postprocess.padim.FilterDetection(threshold = threshold)([score, mask])

model = tf.keras.Model(x, filtered_out)
[p.shape for p in model.predict(te_pipe, verbose = 0)]

3/3 [==============================] - 2s 77ms/step


[(10, 1), (10, 32, 32, 1)]

In [5]:
x = tf.keras.layers.Input(shape = [*image_shape, 3])
feature = tfdet.model.backbone.wide_resnet50_2(x, weights = "imagenet_v2", indices = [1, 2])
out = tfdet.model.detector.patch_core(feature)
model = tf.keras.Model(x, out)

extracted_feature = model.predict(tr_pipe, verbose = 1)
feature_vector = tfdet.model.train.patch_core.train(extracted_feature)

score, mask = tfdet.model.detector.patch_core(feature, feature_vector, image_shape = image_shape, k = 9)
model = tf.keras.Model(x, score)
score_pred = model.predict(val_pipe, verbose = 1)
threshold = tfdet.util.get_threshold(val_y, score_pred)
filtered_out = tfdet.model.postprocess.patch_core.FilterDetection(threshold = threshold)([score, mask])

model = tf.keras.Model(x, filtered_out)
[p.shape for p in model.predict(te_pipe, verbose = 0)]

3/3 [==============================] - 1s 25ms/step


greedy sampling top-k center: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


3/3 [==============================] - 2s 29ms/step


[(10, 1), (10, 32, 32, 1)]

In [6]:
x = tf.keras.layers.Input(shape = [*image_shape, 3])
feature = tfdet.model.backbone.wide_resnet50_2(x, weights = "imagenet_v2", indices = [0, 1, 2, 3])
out = tfdet.model.detector.spade(feature)
model = tf.keras.Model(x, out)

feature_vector = model.predict(tr_pipe, verbose = 1)

score, mask = tfdet.model.detector.spade(feature, feature_vector, image_shape = image_shape, k = 50) #align memory_reduce with train in test
model = tf.keras.Model(x, score)
score_pred = model.predict(val_pipe, verbose = 1)
threshold = tfdet.util.get_threshold(val_y, score_pred)
filtered_out = tfdet.model.postprocess.spade.FilterDetection(threshold = threshold)([score, mask])

model = tf.keras.Model(x, filtered_out)
[p.shape for p in model.predict(te_pipe, verbose = 0)]

3/3 [==============================] - 2s 44ms/step


[(10, 1), (10, 32, 32, 1)]